# resnetv2_50x1_bitm (X1)

In [3]:
import torch
import sys
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")
print("Python path:", sys.executable)


Torch version: 2.5.1+cu121
CUDA available: True
Device name: NVIDIA GeForce RTX 3060
Python path: c:\Users\Admin\anaconda3\envs\tf\python.exe


In [6]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import torch
import torch.nn as nn
from timm import create_model
import torch.optim as optim
import numpy as np

# ==== Cài đặt chung ====
data_dir = r"C:\Users\Admin\Documents\Python Project\Res conn 2025\final_data\not_seg"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")
# ==== Tiền xử lý ====
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load dataset với transform mặc định (chưa có augmentation) để chia dataset
full_dataset = datasets.ImageFolder(root=data_dir, transform=val_test_transform)

# ==== Chia dataset thành train, val, test ====
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

total_size = len(full_dataset)
train_size = int(train_ratio * total_size)
val_size = int(val_ratio * total_size)
test_size = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

# Vì train_dataset, val_dataset, test_dataset là subset nên phải thay đổi transform thủ công:
train_dataset.dataset.transform = train_transform
val_dataset.dataset.transform = val_test_transform
test_dataset.dataset.transform = val_test_transform

# ==== DataLoader ====
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# ==== Mô hình Resnet50v2 ====
model = create_model('resnetv2_50x1_bitm', pretrained=True, num_classes=len(full_dataset.classes))
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00003, weight_decay=0.01)

# ==== EarlyStopping ====
class EarlyStopping:
    def __init__(self, patience=15, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = np.inf
        self.counter = 0

    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            return False
        else:
            self.counter += 1
            if self.counter >= self.patience:
                return True
            return False

early_stopping = EarlyStopping(patience=3, min_delta=0.001)

# ==== Training loop ====
epochs = 500
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    train_loss = running_loss / len(train_loader)

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    val_loss /= len(val_loader)

    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

    if early_stopping(val_loss):
        print(f"Early stopping at epoch {epoch+1}")
        break


c:\Users\Admin\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Using device: cuda


c:\Users\Admin\anaconda3\envs\tf\lib\site-packages\timm\models\_factory.py:126: UserWarning: Mapping deprecated model name resnetv2_50x1_bitm to current resnetv2_50x1_bit.goog_in21k_ft_in1k.
  model = create_fn(


Epoch 1, Train Loss: 0.3314, Val Loss: 0.2088
Epoch 2, Train Loss: 0.1468, Val Loss: 0.1882
Epoch 3, Train Loss: 0.1140, Val Loss: 0.1955
Epoch 4, Train Loss: 0.0983, Val Loss: 0.2126
Epoch 5, Train Loss: 0.0835, Val Loss: 0.2282
Early stopping at epoch 5


In [7]:
from sklearn.metrics import f1_score, precision_score, recall_score

model.eval()
correct = 0
total = 0

all_preds = []
all_labels = []

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        _, predicted = torch.max(outputs, 1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Accuracy
accuracy = 100 * correct / total
print(f"✅ Accuracy on test set: {accuracy:.2f}%")

# F1, Precision, Recall
f1 = f1_score(all_labels, all_preds, average='weighted')
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')

print(f"📊 F1 Score (weighted): {f1:.4f}")
print(f"📌 Precision (weighted): {precision:.4f}")
print(f"📈 Recall (weighted): {recall:.4f}")


✅ Accuracy on test set: 92.13%
📊 F1 Score (weighted): 0.9188
📌 Precision (weighted): 0.9185
📈 Recall (weighted): 0.9213


# resnetv2_50x3_bitm (X3)

In [6]:
# ==== Mô hình Resnet50v2 ====
model2 = create_model('resnetv2_50x3_bitm', pretrained=True, num_classes=len(full_dataset.classes))
model2.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model2.parameters(), lr=0.0001, weight_decay=0.01)

# ==== EarlyStopping ====
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = np.inf
        self.counter = 0

    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            return False
        else:
            self.counter += 1
            if self.counter >= self.patience:
                return True
            return False

early_stopping = EarlyStopping(patience=5, min_delta=0.001)

# ==== Training loop ====
epochs = 50
for epoch in range(epochs):
    model2.train()
    running_loss = 0.0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model2(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    train_loss = running_loss / len(train_loader)

    model2.eval()
    val_loss = 0.0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model2(imgs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    val_loss /= len(val_loader)

    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

    if early_stopping(val_loss):
        print(f"Early stopping at epoch {epoch+1}")
        break


c:\Users\Admin\anaconda3\envs\tf\lib\site-packages\timm\models\_factory.py:126: UserWarning: Mapping deprecated model name resnetv2_50x3_bitm to current resnetv2_50x3_bit.goog_in21k_ft_in1k.
  model = create_fn(
c:\Users\Admin\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--timm--resnetv2_50x3_bit.goog_in21k_ft_in1k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: 

Epoch 1, Train Loss: 0.3973, Val Loss: 0.2092
Epoch 2, Train Loss: 0.2020, Val Loss: 0.1989
Epoch 3, Train Loss: 0.1784, Val Loss: 0.2085
Epoch 4, Train Loss: 0.1410, Val Loss: 0.2346
Epoch 5, Train Loss: 0.1380, Val Loss: 0.2434
Epoch 6, Train Loss: 0.1084, Val Loss: 0.2403
Epoch 7, Train Loss: 0.1044, Val Loss: 0.2112
Early stopping at epoch 7


In [7]:
model2.eval()
correct = 0
total = 0
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model2(imgs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {100 * correct / total:.2f}%")


Accuracy on test set: 92.39%
